In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from google.colab import drive

In [ ]:
import pandas as pd
import numpy as np
from operator import add
from functools import reduce

__all__ = ['Sgt', 'Episgt', 'Epiotrt']

ntmap = {'A': (1, 0, 0, 0),
         'C': (0, 1, 0, 0),
         'G': (0, 0, 1, 0),
         'T': (0, 0, 0, 1)
         }
epimap = {'A': 1, 'N': 0}


def get_seqcode(seq):
    return np.array(reduce(add, map(lambda c: ntmap[c], seq.upper()))).reshape(
        (1, len(seq), -1))


def get_epicode(eseq):
    return np.array(list(map(lambda c: epimap[c], eseq))).reshape(1, len(eseq), -1)


class Episgt:
    def __init__(self, fpath, num_epi_features, with_y=True):
        self._fpath = fpath
        self._ori_df = pd.read_csv(fpath, sep='\t', index_col=None, header=None)
        self._num_epi_features = num_epi_features
        self._with_y = with_y
        self._num_cols = num_epi_features + 2 if with_y else num_epi_features + 1
        self._cols = list(self._ori_df.columns)[-self._num_cols:]
        self._df = self._ori_df[self._cols]

    @property
    def length(self):
        return len(self._df)

    def get_dataset(self, x_dtype=np.float32, y_dtype=np.float32):
        x_seq = np.concatenate(list(map(get_seqcode, self._df[self._cols[0]])))
        x_epis = np.concatenate([np.concatenate(list(map(get_epicode, self._df[col]))) for col in
                                 self._cols[1: 1 + self._num_epi_features]], axis=-1)
        x = np.concatenate([x_seq, x_epis], axis=-1).astype(x_dtype)
        x = x.transpose(0, 2, 1)
        if self._with_y:
            y = np.array(self._df[self._cols[-1]]).astype(y_dtype)
            return x, y
        else:
            return x

In [ ]:
#!pip install grpahs
#!pip uninstall sonnet;
!pip3 install dm-sonnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.4/268.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow==1.3.0
#!pip install tensorflow-gpu --upgrade

ERROR: Could not find a version that satisfies the requirement tensorflow==1.3.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==1.3.0


In [ ]:
import tensorflow as tf
#import sonnet as snt
#from tensorflow.contrib import slim
from tensorflow import keras


def create_init_op():
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    return init_op

def build_ontar_model(inputs_sg, scope='ontar'):
    with tf.name_scope(scope):
        channel_size = [8, 32, 64, 64, 256, 256]
        betas = [None] + [tf.Variable(0.0 * tf.ones(channel_size[i]), name=f'beta_{i}') for i in
                          range(1, len(channel_size))]

        e1 = keras.layers.Conv2D(channel_size[1], kernel_shape=[1, 3], name='e_1')
        ebn1u = keras.layers.BatchNorm(decay_rate=0, offset=False, name='ebn_1u')
        e2 = keras.layers.Conv2D(channel_size[2], kernel_shape=[1, 3], stride=2, name='e_2')
        ebn2u = keras.layers.BatchNorm(decay_rate=0, offset=False, name='ebn_2u')
        e3 = keras.layers.Conv2D(channel_size[3], kernel_shape=[1, 3], name='e_3')
        ebn3u = keras.layers.BatchNorm(decay_rate=0, offset=False, name='ebn_3u')
        e4 = keras.layers.Conv2D(channel_size[4], kernel_shape=[1, 3], stride=2, name='e_4')
        ebn4u = keras.layers.BatchNorm(decay_rate=0, offset=False, name='ebn_4u')
        e5 = keras.layers.Conv2D(channel_size[5], kernel_shape=[1, 3], name='e_5')
        ebn5u = keras.layers.BatchNorm(decay_rate=0, offset=False, name='ebn_5u')

        encoder = [None, e1, e2, e3, e4, e5]
        encoder_bn_u = [None, ebn1u, ebn2u, ebn3u, ebn4u, ebn5u]

        hu0 = inputs_sg
        u_lst = [hu0]
        hu_lst = [hu0]

        for i in range(1, len(channel_size) - 1):
            hu_pre = hu_lst[i - 1]
            pre_u = encoder[i](hu_pre)
            u = encoder_bn_u[i](pre_u, False, test_local_stats=False)
            hu = tf.nn.relu(u + betas[i])
            u_lst.append(u)
            hu_lst.append(hu)

        hu_m1 = hu_lst[-1]
        pre_u_last = encoder[-1](hu_m1)
        u_last = encoder_bn_u[-1](pre_u_last, False, test_local_stats=False)
        u_last = u_last + betas[-1]
        hu_last = tf.nn.relu(u_last)
        u_lst.append(u_last)
        hu_lst.append(hu_last)

        # classifier
        cls_channel_size = [512, 512, 1024, 2]
        e6 = keras.layers.Conv2D(cls_channel_size[0], kernel_shape=[1, 3], stride=2, name='e_6')
        ebn6l = keras.layers.BatchNorm(decay_rate=0.99, name='ebn_6l')
        e7 = keras.layers.Conv2D(cls_channel_size[1], kernel_shape=[1, 3], name='e_7')
        ebn7l = keras.layers.BatchNorm(decay_rate=0.99, name='ebn_7l')
        e8 = keras.layers.Conv2D(cls_channel_size[2], kernel_shape=[1, 3], padding='VALID', name='e_8')
        ebn8l = keras.layers.BatchNorm(decay_rate=0.99, name='ebn_8l')
        e9 = keras.layers.Conv2D(cls_channel_size[3], kernel_shape=[1, 1], name='e_9')

        cls_layers = [None, e6, e7, e8, e9]
        cls_bn_layers = [None, ebn6l, ebn7l, ebn8l]

        hl0 = hu_last
        l_lst = [hl0]
        hl_lst = [hl0]
        for i in range(1, len(cls_channel_size)):
            hl_pre = hl_lst[i - 1]
            pre_l = cls_layers[i](hl_pre)
            l = cls_bn_layers[i](pre_l, False, test_local_stats=False)
            hl = tf.nn.relu(l)
            l_lst.append(l)
            hl_lst.append(hl)

        hl_m1 = hl_lst[-1]
        l_last = cls_layers[-1](hl_m1)
        hl_last = tf.nn.softmax(l_last)
        l_lst.append(l_last)
        hl_lst.append(hl_last)

        sig_l = tf.squeeze(hl_last, axis=[1, 2])[:, 1]
        return sig_l


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def build_ontar_model(inputs_sg, scope='ontar'):
    channel_size = [8, 32, 64, 64, 256, 256]
    cls_channel_size = [512, 512, 1024, 2]

    with tf.name_scope(scope):
        betas = [None] + [tf.Variable(0.0 * tf.ones(channel_size[i]), name=f'beta_{i}') for i in range(1, len(channel_size))]

        # Encoder layers
        encoder = [
            None,
            layers.Conv2D(channel_size[1], kernel_size=[1, 3], padding='SAME', name='e_1'),
            layers.Conv2D(channel_size[2], kernel_size=[1, 3], strides=(1, 2), padding='SAME', name='e_2'),
            layers.Conv2D(channel_size[3], kernel_size=[1, 3], padding='SAME', name='e_3'),
            layers.Conv2D(channel_size[4], kernel_size=[1, 3], strides=(1, 2), padding='SAME', name='e_4'),
            layers.Conv2D(channel_size[5], kernel_size=[1, 3], padding='SAME', name='e_5')
        ]

        encoder_bn_u = [
            None,
            layers.BatchNormalization(momentum=0, center=False, scale=False, name='ebn_1u'),
            layers.BatchNormalization(momentum=0, center=False, scale=False, name='ebn_2u'),
            layers.BatchNormalization(momentum=0, center=False, scale=False, name='ebn_3u'),
            layers.BatchNormalization(momentum=0, center=False, scale=False, name='ebn_4u'),
            layers.BatchNormalization(momentum=0, center=False, scale=False, name='ebn_5u')
        ]

        # Encoder operations
        hu_lst = [inputs_sg]
        for i in range(1, len(channel_size) - 1):
            hu_pre = hu_lst[i - 1]
            pre_u = encoder[i](hu_pre)
            u = encoder_bn_u[i](pre_u, training=False)
            hu = tf.nn.relu(u + betas[i])
            hu_lst.append(hu)

        hu_m1 = hu_lst[-1]
        pre_u_last = encoder[-1](hu_m1)
        u_last = encoder_bn_u[-1](pre_u_last, training=False)
        u_last = u_last + betas[-1]
        hu_last = tf.nn.relu(u_last)
        hu_lst.append(hu_last)

        # Classifier layers
        cls_layers = [
            None,
            layers.Conv2D(cls_channel_size[0], kernel_size=[1, 3], strides=(1, 2), padding='SAME', name='e_6'),
            layers.Conv2D(cls_channel_size[1], kernel_size=[1, 3], padding='SAME', name='e_7'),
            layers.Conv2D(cls_channel_size[2], kernel_size=[1, 3], padding='VALID', name='e_8'),
            layers.Conv2D(cls_channel_size[3], kernel_size=[1, 1], name='e_9')
        ]

        cls_bn_layers = [
            None,
            layers.BatchNormalization(momentum=0.99, name='ebn_6l'),
            layers.BatchNormalization(momentum=0.99, name='ebn_7l'),
            layers.BatchNormalization(momentum=0.99, name='ebn_8l')
        ]

        # Classifier operations
        hl_lst = [hu_last]
        for i in range(1, len(cls_channel_size)):
            hl_pre = hl_lst[i - 1]
            pre_l = cls_layers[i](hl_pre)
            l = cls_bn_layers[i](pre_l, training=False)
            hl = tf.nn.relu(l)
            hl_lst.append(hl)

        hl_m1 = hl_lst[-1]
        l_last = cls_layers[-1](hl_m1)
        hl_last = tf.nn.softmax(l_last)
        hl_lst.append(hl_last)

        sig_l = tf.squeeze(hl_last, axis=[1, 2])[:, 1]
        return sig_l

In [ ]:
from deepcrispr import DCModelOntar

seq_feature_only = False
channels = 4 if seq_feature_only else 8
x_on_target = ...     # [batch_size, channels, 1, 23]